In [1]:

#test plan history
plan_columns = ["Data", "Version", "Name", "Description", "Status", "Owner"]
plan_histroy = [
    "Data=07/10/2024; Version=0.5; Name=aaa; Description=bbb; Status=ccc; Owner=ddd",
    "Data=07/11/2024; Version=0.6; Name=aaa; Description=bbb; Status=ccc; Owner=ddd",
    "Data=07/12/2024; Version=0.7; Name=aaa; Description=bbb; Status=ccc; Owner=ddd",
]


In [1]:
#source_code
source_code = """

UINT32 PCISetLinkSpeed(int index, TaskContext *ctx, UINT32 speed) {
    UINT32 status;
    UINT32 offset;
    UINT32 size;

    if (speed < 1 || speed > 5) {
        std::cout << "Invalid link speed value. Valid values are 1, 2, 3, 4, 5" << std::endl;
        return IOCTL_STATUS_INVALID_PARAMETER;
    }

    do {
        try {
            BDF bdf = ctx->bdf;
            UINT16 LinkControl2Register = ctx->LinkControl2Register;

            LinkControl2Register &= ~BIT_3_0_MASK;
            switch (speed) {
            case 1:
                LinkControl2Register |= PCIE_LINK_SPEED_2_5;
                break;
            case 2:
                LinkControl2Register |= PCIE_LINK_SPEED_5_0;
                break;
            case 3:
                LinkControl2Register |= PCIE_LINK_SPEED_8_0;
                break;
            case 4:
                LinkControl2Register |= PCIE_LINK_SPEED_16_0;
                break;
            case 5:
                LinkControl2Register |= PCIE_LINK_SPEED_32_0;
                break;
            default:
                std::cout << "Invalid TargetLinkSpeed " << speed << "\n";
                status = IOCTL_STATUS_FAIL;
                break;
            }

            offset = ctx->CapabilitiesPtr + PCIE_CAP_LINK_CONTROL2_OFFSET;
            size = 2;

            // Write to Link Control 2 Register with new speed
            status = HSIOControlPCIConfigWrite(ctx->devHandle.get(), bdf.segment, bdf.bus, bdf.device, bdf.function,
                                               (PCHAR)&LinkControl2Register, offset, size);
            if (status != IOCTL_STATUS_SUCCESS) {
                std::cout << "Writing Config Space error = 0x" << std::hex << status << "  s:b:d:f = " << bdf.segment
                          << ":" << static_cast<int>(bdf.bus) << ":" << static_cast<int>(bdf.device) << ":"
                          << static_cast<int>(bdf.function) << "\n";
            }

        } catch (const std::exception &e) {
            // Handle standard exceptions
            std::cerr << "Standard exception: " << e.what() << std::endl;
            status = IOCTL_STATUS_FAIL;
        } catch (...) {
            // Handle all other exceptions
            std::cerr << "Unknown exception occurred." << std::endl;
            status = IOCTL_STATUS_FAIL;
        }
    } while (false);

    ctx->lastStatus = status;
    ctx->index = index;

    return status;
}

"""


to access the openAI API, you need an API key which you can get it from  https://platform.openai.com/api-keys
once you have a key then you can either save it as an environment variable or hard code in as needed from simplicity

In [2]:
import os
import time
import openai

import streamlit as st
from decouple import config
from dotenv import load_dotenv
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Load the OpenAI API key from the environment variables
try:
    load_dotenv(override=True)
    # os.environ["OPENAI_API_KEY"] = "your key"
    openai.api_key = os.environ.get("OPENAI_API_KEY")
    if not openai.api_key:
        raise ValueError("OPENAI_API_KEY is not set in the environment variables.")
except Exception as e:
    print(f"Error loading environment variables or setting OpenAI API key: {e}")


initialize OpenAI model

In [3]:
# openAI model name
# https://platform.openai.com/docs/models
#openAI_model="gpt-4"
#openAI_model="gpt-4-32k"
#openAI_model="gpt-4-turbo"
openAI_model="gpt-4o"

In [4]:
from langchain_openai import ChatOpenAI

try:
    prompt = PromptTemplate(
        input_variables=["chat_history", "question"],
            template="""prepare code review comments based on the code asked by the user, 
            and suggest ways to refactor and improve code quality based on the context below.

            Context:  You are code reviewer who has a high level of experience in
             development and code review for the C language or C++ language or Python language.  

            A code reviewer's main responsibility is to ensure the codebase is consistent and maintainable. 
            A code reviewer has ownership and responsibility over the code they are reviewing.
            you do this by following best practices and guidelines, and by providing feedback to the developer.

            - Double-checking the author's work
            - Identifying improvements
            - Considering how the change fits into the codebase
            - Checking the clarity of the title and description
            - Verifying the correctness of the code
            - Testing coverage
            - Checking for functionality changes
            - Confirming that the code follows coding guides and best practices
            - The code is well-designed.
            - The functionality is good for the users of the code.
            - Any UI changes are sensible and look good.
            - Any parallel programming is done safely.
            - The code isn’t more complex than it needs to be.
            - The developer isn’t implementing things they might need in the future but don’t know they need now.
            - Code has appropriate unit tests.
            - Tests are well-designed.
            - The developer used clear names for everything.
            - Comments are clear and useful, and mostly explain why instead of what.
            - Code is appropriately documented (generally in G3doc which is a documentation system developed by Google. It is designed to make it easier for teams to create, manage, and share documentation within the company).
            - The code conforms to our style guides.
            - Recognize common coding errors and suggest fixes.

            when you write code review comments, follow these guidelines.
            - Be kind and respectful.
            - Explain your reasoning.
            - Balance giving explicit directions with just pointing out problems and letting the developer decide.
            - Encourage developers to simplify code or add code comments instead of just explaining the complexity to you.
            - labeling the severity of your comments, differentiating required changes from guidelines or suggestions.

            you need to write code review comments based on the code asked by the user, and suggest ways to refactor and improve code quality,
            enhance performance, address security concerns, and align with the best practices and guidelines.

            you need to rewrite code based on the code review feedback you have provided. Provide an easy way for developers to copy and run the code. 

        chat_history: {chat_history}

        Question: {question}

        Answer:"""
    )

    llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model_name=openAI_model)

    memory = ConversationBufferWindowMemory(memory_key="chat_history", k=4)
    llm_chain = LLMChain(
        llm=llm,
        memory=memory,
        prompt=prompt
    )
except Exception as e:
    print(f"Failed to initialize ChatOpenAI : {e}")


In [5]:
messages = [
                {"role": "assistant", "content": "Hello there"}
]


run review

In [6]:
user_prompt = source_code

messages.append({"role": "user", "content": user_prompt})
ai_response = llm_chain.predict(question=user_prompt)
messages.append({"role": "assistant", "content": ai_response})

print(ai_response)

### Code Review Comments

#### Required Changes

1. **Memory Deallocation Safety**:
   - Ensure that memory deallocation and unmapping are performed safely to prevent potential memory leaks or undefined behavior.

2. **Code Safety**:
   - Add checks for null pointers before dereferencing them to avoid potential crashes.

3. **Thread Safety**:
   - Ensure that the function is thread-safe if it can be called from multiple threads.

4. **Function Documentation**:
   - Document the function properly to explain what it does, its parameters, and its return value.

5. **Consistency**:
   - Ensure the code follows consistent coding standards, such as naming conventions and indentation.

#### Suggestions

1. **Simplify Code**:
   - Simplify the while loop condition for better readability.

2. **Logging**:
   - Improve the logging statement to be more consistent and informative. 

3. **Error Handling**:
   - Add error handling to manage cases where `MmUnmapIoSpace` or `MmFreeNonCachedMemory` mig